# Análisis Exploratorio de Datos (EDA)
## Proyecto: Sistema de Dermatoscopia Inteligente

**Estudiantes:**
* Cory Bryan Contreras Cedillo
* Andrés Hernández Ruíz
* Eduardo Rojas Pérez

### 1. Introducción
El diagnóstico temprano del melanoma y otras patologías cutáneas es determinante para la tasa de supervivencia del paciente. Sin embargo, la implementación de sistemas de Diagnóstico Asistido por Computadora (CAD) suele depender de servidores externos (Cloud Computing), lo que limita su uso en zonas remotas o con conectividad limitada.

Este proyecto propone el desarrollo de un modelo de **Deep Learning eficiente (Edge AI)**, capaz de ejecutarse en dispositivos de recursos limitados (Raspberry Pi), manteniendo una sensibilidad diagnóstica aceptable. Para ello, se utilizan conjuntos de datos biomédicos estandarizados (`MedMNIST`) que permiten validar arquitecturas ligeras.

### 2. Planteamiento del Problema
El desarrollo del modelo enfrenta dos desafíos técnicos críticos que deben ser cuantificados antes del entrenamiento:
1.  **Escasez y Resolución de Datos:** Las imágenes de entrada poseen una resolución nativa de baja dimensionalidad ($28 \times 28$ píxeles), lo que obliga a evaluar si la información visual es suficiente para la extracción de características o si se requiere *Upsampling*.
2.  **Desbalance de Clases (Class Imbalance):** En la práctica dermatológica, las lesiones benignas (como los nevus) son significativamente más frecuentes que los melanomas. Se hipnotiza que el dataset `DermaMNIST` refleja este sesgo, lo cual, de no corregirse, provocará que la red neuronal optimice su precisión ignorando los casos malignos (paradoja de la exactitud).

### 3. Objetivos del Análisis (EDA)
* **Validar la integridad dimensional:** Asegurar que los tensores de entrada $(N, H, W, C)$ sean compatibles con la arquitectura de CNN propuesta.
* **Establecer una Línea Base (Baseline):** Utilizar `PathMNIST` como dataset de control para validar el flujo de trabajo.
* **Cuantificar el Desbalance:** Calcular matemáticamente los pesos de clase ($W_c$) necesarios para implementar una función de pérdida ponderada (*Weighted Cross-Entropy Loss*).

### 4. Código Bloque 1

**1: Importación de librerías y definición de rutas**

In [1]:
import os       # Para manejo de rutas agnóstico al sistema operativo (Win/Linux)
import sys      # Para manipulación de variables del sistema
import numpy as np              # Procesamiento numérico y matricial (Tensores)
import pandas as pd             # Estructuras de datos para conteos y estadística
import matplotlib.pyplot as plt # Visualización base
import seaborn as sns           # Visualización estadística de alto nivel

*Justificación:* Establece una arquitectura de rutas relativas agnóstica al sistema operativo. Esto garantiza que el código sea reproducible tanto en estaciones de trabajo de alto rendimiento (GPU) como en el dispositivo final (Raspberry Pi) sin modificar el código fuente.

**2: Configuración de Estilo "Paper Académico"**

In [2]:
sns.set_theme(style="whitegrid") # Fondo con rejilla para facilitar lectura de valores
plt.rcParams['figure.figsize'] = (12, 6) # Tamaño estándar para visualización HD
plt.rcParams['font.size'] = 11  # Tamaño de letra legible

*Justificación:* Implementa el principio de ingeniería *DRY (Don't Repeat Yourself)*. Se crea una función modular para la ingesta de datos `.npz`, permitiendo manejar excepciones (archivos corruptos o inexistentes) de manera robusta antes de que afecten el flujo de memoria.

**3: Definición de Rutas Relativas**

In [3]:
BASE_DIR = os.path.dirname(os.getcwd()) 
DATA_DIR = os.path.join(BASE_DIR, "assets", "datasets")

# NOTA: os.getcwd() nos da la carpeta actual (notebooks)
# os.path.dirname(...) nos lleva un nivel arriba (raíz del proyecto)

*Justificación:* Validación *Human-in-the-loop*. Antes de procesar números, se visualizan muestras aleatorias con sus etiquetas clínicas reales. Esto permite detectar anomalías visuales (ruido, artefactos) y confirmar si la resolución es suficiente para distinguir patrones a simple vista.

**4: Verificación de Integridad del Entorno**

In [4]:
print(f"Entorno Configurado Correctamente.")
print(f"Directorio Raíz del Proyecto: {BASE_DIR}")
print(f"Directorio de Datos Esperado: {DATA_DIR}")

# Verificamos si la carpeta assets existe
if os.path.exists(DATA_DIR):
    archivos = os.listdir(DATA_DIR)
    print(f"Archivos encontrados: {archivos}")
else:
    print(f"ALERTA CRÍTICA: No se encuentra la carpeta 'assets/datasets'. Revisa la estructura.")

Entorno Configurado Correctamente.
Directorio Raíz del Proyecto: c:\Proyecto_integrador
Directorio de Datos Esperado: c:\Proyecto_integrador\assets\datasets
Archivos encontrados: ['dermamnist.npz', 'pathmnist.npz']


*Justificación:* Es el núcleo del EDA. Se generan histogramas de frecuencia por clase. Esta evidencia gráfica y numérica es la prueba formal de la necesidad de aplicar técnicas de corrección de sesgo (como *Weighted Loss* o *Data Augmentation*) en la siguiente fase del proyecto.